# Aula 3 (Parte 2) - Eliminação Gaussiana

### BLAS - Basic Linear Algebra Subroutines
$$ αx, x+y, x^Ty = <x,y> $$
$$ Ax, Lx=b, Ux=b $$
$$ L^{-1}b, U^{-1}b $$

### 3 níveis de operações

$$ z = αx + y => z_{i} = αx_{i} + y_{i} $$
<div align = "center"> <b>Custo:</b> n * e n + = 2n </div>
$$ β = x^Ty => β = \sum_{i=1}^{n}x_{i}y_{i} $$
<div align = "center"> <b>Custo:</b> n * e n-1 + = 2n-1 </div>
<br>
Operações lineares, <b>O(n) ≤ Cn</b>, BLAS nível 1

========================================================================================================================

$$ y = Ax => y_{i} = \sum_{j=1}^{n}2ij = (a_{i}.)^Tx_{j} $$
Operações quadráticas, <b>(2n - 1)m = 2mn - m</b>, BLAS nível 2

========================================================================================================================

$$ L = \begin{bmatrix} 
    l_{11} & 0 & 0 \\
    \vdots & \ddots & 0\\
    l_{n1} & \dots & l_{nn} 
    \end{bmatrix} ,
    U = \begin{bmatrix} 
    u_{11} & u_{12} & \dots \\
    0 & \ddots & \vdots\\
    0 & 0 & u_{nn} 
    \end{bmatrix}$$
BLAS nível 2

========================================================================================================================

$$ C = AB, A\in R^{mxn}, B\in R^{nxp} $$
$$ C \in R^{mxp} $$
$$ c_{ij} = \sum_{k=1}^{n}a_{ik}b_{kj} $$
<b>mpO(n) = O(mpn)</b>, BLAS nível 3

In [2]:
A = rand(3, 5)
B = rand(5, 4)
A * B

3×4 Array{Float64,2}:
 1.04364   0.940909  1.07188   1.08917
 0.738065  0.575652  0.784234  0.29438
 1.4533    1.18482   0.825507  1.08822

In [3]:
C = zeros(3, 4)
for i = 1:3, j = 1:4
    C[i, j] = sum(A[i, k] * B[k, j] for k = 1:5)
end
C

3×4 Array{Float64,2}:
 1.04364   0.940909  1.07188   1.08917
 0.738065  0.575652  0.784234  0.29438
 1.4533    1.18482   0.825507  1.08822

### Esparsidade de Matrizes

$$  \begin{bmatrix} 
    2 & -1 & & & & 0 \\
    -1 & 2 & -1 & & &\\
     & -1 & 2 & -1 & &\\
     & & \ddots & \ddots & \ddots\\ 
     & & & \ddots & \ddots & -1\\ 
     0 & & & & -1 & 2
    \end{bmatrix}_{nxn}
    \begin{bmatrix}
    x_{1}\\ \vdots\\ \vdots\\ 
    \vdots\\ \vdots\\ x_{n}
    \end{bmatrix} = \dots $$

Número de não zeros (nnz) = <b>n + 2(n - 1) = 3n - 2</b>

In [7]:
n = 100000
Float64.size * (3n - 2) / 1024 / 1024, Float64.size * n^2 / 1024 / 1024

(2.2888031005859375, 76293.9453125)

In [8]:
using Pkg
pkg"add BenchmarkTools"

   Updating registry at `C:\Users\rodri\.julia\registries\General`
   Updating registry at `C:\Users\rodri\.julia\registries\JuliaComputingRegistry`
  Resolving package versions...
  Installed BenchmarkTools ─ v0.5.0
Updating `C:\Users\rodri\.julia\environments\JuliaPro_v1.5.3-1\Project.toml`
  [6e4b80f9] + BenchmarkTools v0.5.0
Updating `C:\Users\rodri\.julia\environments\JuliaPro_v1.5.3-1\Manifest.toml`
  [6e4b80f9] + BenchmarkTools v0.5.0


In [9]:
using BenchmarkTools

┌ Info: Precompiling BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1278


In [11]:
function multAx1(y, A, x)
    m, n = size(A)
    for i = 1:m
        y[i] = 0.0
        for j = 1:n
            y[i] += A[i,j] * x[j]
        end
    end
end

function multAx2(y, A, x)
    m, n = size(A)
    for i = 1:m
        y[i] = 0.0
    end
    for j = 1:n
        for i = 1:m
            y[i] += A[i, j] * x[j]
        end
    end
end

multAx2 (generic function with 1 method)

In [12]:
n = 1000
A = rand(n, n)
x = rand(n)
x = rand(n);

In [14]:
y1 = zeros(n)
y2 = zeros(n)
multAx1(y1, A, x)
multAx2(y2, A, x)
y1 ≈ y2 ≈ A * x # provando que y1, y2 e A*x são aproximadamente, para poder comparar as duas funções

true

In [15]:
y = zeros(n)
@benchmark multAx1(y, $A, $x)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     1.506 ms (0.00% GC)
  median time:      1.622 ms (0.00% GC)
  mean time:        1.712 ms (0.00% GC)
  maximum time:     3.990 ms (0.00% GC)
  --------------
  samples:          2919
  evals/sample:     1

In [17]:
y = zeros(n)
@benchmark multAx2(y, $A, $x) 
# Essa maneira foi mais rápida que a anterior, por conseguir acessar os elementos do vetor numa ordem preferível

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     702.500 μs (0.00% GC)
  median time:      727.499 μs (0.00% GC)
  mean time:        742.714 μs (0.00% GC)
  maximum time:     4.265 ms (0.00% GC)
  --------------
  samples:          6727
  evals/sample:     1

In [19]:
pkg"add SparseArrays"

  Resolving package versions...
No Changes to `C:\Users\rodri\.julia\environments\JuliaPro_v1.5.3-1\Project.toml`
No Changes to `C:\Users\rodri\.julia\environments\JuliaPro_v1.5.3-1\Manifest.toml`


In [20]:
using SparseArrays

In [21]:
A = sprand(10, 10, 0.3)

10×10 SparseMatrixCSC{Float64,Int64} with 33 stored entries:
  [1 ,  1]  =  0.565155
  [7 ,  1]  =  0.428156
  [7 ,  2]  =  0.74719
  [8 ,  2]  =  0.241782
  [10,  2]  =  0.830831
  [1 ,  3]  =  0.418315
  [3 ,  3]  =  0.234554
  [7 ,  3]  =  0.0694216
  [9 ,  3]  =  0.568618
  [2 ,  4]  =  0.142559
  [3 ,  4]  =  0.834241
  [7 ,  4]  =  0.252775
  ⋮
  [8 ,  6]  =  0.696957
  [9 ,  6]  =  0.480471
  [10,  6]  =  0.379286
  [2 ,  7]  =  0.95539
  [9 ,  7]  =  0.983426
  [8 ,  8]  =  0.293096
  [6 ,  9]  =  0.860879
  [7 ,  9]  =  0.604503
  [9 ,  9]  =  0.907543
  [10,  9]  =  0.119829
  [3 , 10]  =  0.280014
  [6 , 10]  =  0.108418
  [8 , 10]  =  0.538494

In [22]:
rows, cols, vals = findnz(A)

([1, 7, 7, 8, 10, 1, 3, 7, 9, 2  …  2, 9, 8, 6, 7, 9, 10, 3, 6, 8], [1, 1, 2, 2, 2, 3, 3, 3, 3, 4  …  7, 7, 8, 9, 9, 9, 9, 10, 10, 10], [0.5651550316590224, 0.428155862882325, 0.7471899736276217, 0.24178165837320553, 0.830830916863943, 0.4183151471416675, 0.23455364075445395, 0.0694216498013982, 0.5686184565745291, 0.14255865249758637  …  0.9553896799217285, 0.9834263289721739, 0.2930961000168908, 0.860878617480991, 0.604503085254674, 0.9075428658930433, 0.11982878272162667, 0.2800140813748049, 0.10841808582605528, 0.5384938790075391])

## Sistemas Lineares

In [23]:
# Maneira de gerar problemas
A = rand(3, 3)
b = A * ones(3)

3-element Array{Float64,1}:
 2.052806454116383
 1.3488229922393933
 1.3512147111625987

In [24]:
A \ b # Resolve Ax = b

3-element Array{Float64,1}:
 1.0
 0.9999999999999999
 1.0000000000000004

$$ \begin{cases} 
3x_{1} + x_{2} + 2x_{3} = 6 \\ 
-x_{1} + 2x_{2} + x_{3} = 2 \\ 
x_{1} + x_{2} + 4x_{3} = 6
\end{cases} $$
<br>
$$ L_{2} \leftarrow L_{2} + \frac{1}{3}L_{1} = L_{2} - \frac{(-1)}{3}L_{1} $$
$$ L_{2} = (-1, 2, 1; 2) ; L_{1} = (3, 1, 2; 6) $$
$$ L_{2} = L_{2} + \frac{1}{3}L_{1} = (-1, 2, 1; 2) + (1, \frac{1}{3}, \frac{2}{3}; 2) = (0, \frac{8}{3}, \frac{5}{3}; 4) $$
<br>
$$ Matricialmente \hspace{4px} [A \vdots b] \in R^{mx(n+1)} $$
$$ operações \hspace{4px} elementares \hspace{4px} à \hspace{4px} esquerda \to [U \vdots c] $$
$$ Ux = c \hspace{4px} é \hspace{4px} mais \hspace{4px} fácil \hspace{4px} de \hspace{4px} resolver $$

=====================================================================================================================

$$ 1^{a} coluna: L_{1}\hspace{4px}é\hspace{4px}usada\hspace{4px}como\hspace{4px}pivô $$

$$ é\hspace{4px}preciso\hspace{4px}que\hspace{4px}a_{11}\neq 0 $$
<br>
$$ | L_{i} = L_{i} - m_{i1}L_{1} \\ a_{i1} \leftarrow 0 = a_{i1} - m_{i1}a_{i1} \\
   m_{i1} = \frac{a_{i1}}{a_{11}} $$
   
=====================================================================================================================

$$ 2^{a} coluna: L_{2}\hspace{4px}é\hspace{4px}pivô $$

$$ L_{3} \leftarrow L_{3} - m_{32}L_{2} $$
$$ m_{32} = \frac{a_{32}}{a_{22}} $$

## Algorítmo da Eliminação Gaussiana

$$ Para\hspace{4px}j = 1:n $$
$$ Para\hspace{4px}i = j+1:m $$
$$ m_{ij} = a_{ij}/a_{jj} $$
$$ L_{i} \leftarrow L_{i} - m_{ij}L_{j}; $$

In [48]:
A = [3.0 1 2; -1 2 1; 1 1 4]

3×3 Array{Float64,2}:
  3.0  1.0  2.0
 -1.0  2.0  1.0
  1.0  1.0  4.0

In [49]:
b = A * ones(3)

3-element Array{Float64,1}:
 6.0
 2.0
 6.0

In [50]:
m21 = A[2,1] / A[1,1]
m31 = A[3,1] / A[1,1]

# L₂ = A[2, :] , L₃ = A[3, :]
A[2, 2:end] = A[2, 2:end] - m21 * A[1, 2:end]
A[2, 1] = 0.0
A[3, 2:end] = A[3, 2:end] - m31 * A[1, 2:end]
A[3, 1] = 0.0

0.0

In [51]:
b[2] = b[2] - m21 * b[1]
b[3] = b[3] - m31 * b[1]

4.0

In [52]:
A

3×3 Array{Float64,2}:
 3.0  1.0       2.0
 0.0  2.33333   1.66667
 0.0  0.666667  3.33333

In [53]:
b

3-element Array{Float64,1}:
 6.0
 4.0
 4.0

In [54]:
A \ b

3-element Array{Float64,1}:
 1.0
 1.0000000000000002
 0.9999999999999999

In [55]:
m32 = A[3, 2] / A[2, 2]
A[3, 3:end] -= m32 * A[2, 3:end]
A[3, 2] = 0.0
b[3] -= m32 * b[2]

A
# Pode ser que a matriz obtida pela Eliminação Gaussiana não dê o mesmo resultado que a original
# Isso ocorre quando a matriz for mal condicionada

3×3 Array{Float64,2}:
 3.0  1.0      2.0
 0.0  2.33333  1.66667
 0.0  0.0      2.85714

In [56]:
b

3-element Array{Float64,1}:
 6.0
 4.0
 2.8571428571428568

In [57]:
A \ b

3-element Array{Float64,1}:
 1.0
 1.0000000000000002
 0.9999999999999999

## Algorítmo da Eliminação Gaussiana levando em conta a precisão numérica

$$ Para\hspace{4px}j = 1:n $$
$$ Para\hspace{4px}i = j+1:m $$
$$ m_{ij} = a_{ij}/a_{jj} $$
$$ L_{i}(coluna\hspace{4px}j+1:n) = L_{i}(j+1:n) - m_{ij}L(j+1:n); $$

In [77]:
"""
    elim_gauss!(A, b)

Modifica A e b fazendo a eliminação Gaussiana.
"""
function elim_gauss!(A, b)
    m, n = size(A)
    for j = 1:n
        cols = j+1:n
        for i = j+1:m
            mij = A[i, j] / A[j,j]
            A[i, cols] -= mij * A[j, cols]
            A[i, j] = 0.0
            b[i] -= mij * b[j]
        end
    end
end

elim_gauss!

In [96]:
A = rand(5, 5)
#A[:, 3] = A[:, 2] + 2 * A[:, 5]
b = A * ones(5)

5-element Array{Float64,1}:
 0.6996879983590749
 2.4639876205077096
 2.376188644989334
 1.6325105758816598
 2.5609394710657902

In [97]:
A

5×5 Array{Float64,2}:
 0.027839   0.14625     0.0728412  0.263175  0.189583
 0.0499711  0.00832447  0.942036   0.475606  0.98805
 0.806317   0.476486    0.319659   0.302839  0.470887
 0.0474724  0.701355    0.118131   0.58034   0.185212
 0.814596   0.107135    0.167277   0.668382  0.803549

In [98]:
elim_gauss!(A, b)

In [99]:
A

5×5 Array{Float64,2}:
 0.027839   0.14625     0.0728412   0.263175      0.189583
 0.0       -0.254194    0.811286    0.00320572    0.647748
 0.0        0.0       -13.7886     -7.36706     -14.6
 0.0        0.0         0.0        -0.630187     -0.50729
 0.0        0.0         0.0         0.0          -0.0649103

In [100]:
b

5-element Array{Float64,1}:
   0.6996879983590749
   1.2080455402517043
 -35.755724512713726
  -1.1374772762241303
  -0.06491028629416551

In [101]:
A \ b

5-element Array{Float64,1}:
 0.9999999999999691
 1.000000000000049
 0.999999999999941
 0.9999999999999243
 1.0000000000000941